In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

### Reading the Dataset

In [3]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("./datasets/US-Chronic-dataset.csv"))

In [4]:
#df.toPandas().head()

In [5]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

### Selecting Features and Casting it into required Dtype

In [6]:
columns  = df.columns

dataset = df.select(col(columns[3]).cast('string'),
                    col(columns[4]).cast('string'),
                    col(columns[5]).cast('string'),
                    col(columns[6]).cast('float'),
                    col(columns[7]).cast('float'),
                    col(columns[8]).cast('float'),
                    col(columns[9]).cast('string'),
                    col(columns[10]).cast('string'),
                    col(columns[11]).cast('string'),
                    col(columns[12]).cast('string'),
                    col(columns[13]).cast('string'),
                    col(columns[14]).cast('float'),
                    col(columns[15]).cast('float'),
                   )

In [7]:
# inputCols for StringIndexer
string_col = [
    'DataSource','DataValueUnit','DataValueTypeID',
    'QuestionID','LocationID','StratificationCategoryID1',
    'StratificationID1','TopicID'
]

#outputCols for StringIndexer
string_col_output = [
    'DataSourceIndex','DataValueUnitIndex','DataValueTypeIDIndex',
    'QuestionIDIndex','LocationIDIndex','StratificationCategoryID1Index',
    'StratificationID1Index','TopicIDIndex'
]

#inputCols for OneHotEncoder
string_col_encode_input = [
    'DataSourceIndex','DataValueUnitIndex','DataValueTypeIDIndex',
    'QuestionIDIndex','LocationIDIndex','StratificationCategoryID1Index',
    'StratificationID1Index'
]

#outputCols for OneHotEncoder
string_col_encoded = [
    'DataSourceVec','DataValueUnitVec','DataValueTypeIDVec',
    'QuestionIDVec','LocationIDVec','StratificationCategoryID1Vec',
    'StratificationID1Vec'
]

#inputCols for VectorAssembler
features_to_assemble = string_col_encoded + ['DataValue','LowConfidenceLimit','HighConfidenceLimit','Geo_lat','Geo_lon']

### Feature Transformation

In [8]:
indexer = StringIndexer(inputCols= string_col, outputCols=string_col_output)
encoder = OneHotEncoder(inputCols=string_col_encode_input, outputCols=string_col_encoded)
vectorAssembler = VectorAssembler(inputCols=features_to_assemble,
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
scaler = StandardScaler(inputCol="features_norm", outputCol="features_norm_scaled")
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer,scaler])
final_df = pipeline.fit(dataset).transform(dataset)

### Splitting Dataset into training data and test data

In [9]:
(training_data, test_data) = final_df.randomSplit([0.8,0.2],seed=1234)

### Building Model

In [10]:
dtc = DecisionTreeClassifier()
param_grid = (ParamGridBuilder()
              .baseOn({dtc.labelCol : 'TopicIDIndex'})
              .baseOn([dtc.predictionCol, 'pred_dtc'])
              .baseOn([dtc.featuresCol, 'features_norm'])
              .addGrid(dtc.impurity, ['entropy','gini'])
              .addGrid(dtc.maxBins, [32,64])
              .addGrid(dtc.maxDepth, [20,25,30]).build())

evaluator = MulticlassClassificationEvaluator(labelCol= 'TopicIDIndex',predictionCol= 'pred_dtc')

cv = CrossValidator(estimator=dtc, estimatorParamMaps=param_grid, evaluator=evaluator,seed=0,parallelism=2)

### Training Model

In [11]:
cvModel = cv.fit(training_data)

In [12]:
cvModel.avgMetrics

[0.8530009744828864,
 0.9100120508156919,
 0.948191757209137,
 0.8592219013495861,
 0.9119071206541041,
 0.9476137587294395,
 0.6694895371311423,
 0.709777237583297,
 0.7422881343771186,
 0.6588639874944618,
 0.7022554930149069,
 0.734393853379154]

### Evaluating Trained Model using different evaluation metrices

In [13]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "accuracy"})

0.9529846241255637

In [14]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "f1"})

0.9545460913261533

In [15]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "precisionByLabel"})

1.0

In [16]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "recallByLabel"})

0.9586648554240248

### Best Params

In [17]:
cvModel.bestModel.getImpurity()

'entropy'

In [18]:
cvModel.bestModel.getMaxBins()

32

In [19]:
cvModel.bestModel.getMaxDepth()

30

### Saving the Trained Model

In [20]:
path= r'./SavedModels/DTClassifier_model'

In [21]:
cvModel.save(path)

### Loading the Saved Model

In [22]:
#dtc_model = CrossValidatorModel.read().load(path)

### Result Summary

In [1]:
import pandas as pd

summary_={
    'accuracy' : 0.9529846241255637,
    'f1-score' : 0.9545460913261533,
    'precision' : 1.0,
    'recall' : 0.9586648554240248
}

summary = pd.Series(summary_)
summary

accuracy     0.952985
f1-score     0.954546
precision    1.000000
recall       0.958665
dtype: float64